In [ ]:
from dash import Dash,Output,Input,callback,dcc,html,State
import pandas as pd
import yfinance as yf
import numpy as np
import plotly.graph_objs as go
import os
import logging

logging.basicConfig(level=logging.ERROR)

app = Dash()

app.layout = [
    html.Div([html.H1('Stock Dashboard')]),
    html.Div([html.H4('Ticker'),
             dcc.Input(
                 id='ticker-selection',
                 type='text',
                 value='AAPL',
                 style={'color':'#fff','padding':'13px','fontSize':'15px'}
             ),
            html.Button('Submit',id='submit',n_clicks=0),
             ]),
    html.Div([
        html.H4('Dashboard',style={'text_align':'middle'}),
        dcc.Dropdown(
                    options=[
                        {'label':'Margins','value':'margin'},
                        {'label':'Growth','value':'growth'}
                    ],
                    id='dashboard-selection',
                    value='margin',
                    multi=False,
            style={'backgroundColor':'#000','color':'#000'}
        )],
        style={'color':'#fff'}
    ),
    dcc.Graph(id='dashboard-plot')
]

@app.callback(
    Output('dashboard-plot','figure'),
    [Input('submit','n_clicks'),Input('dashboard-selection','value')],
    State('ticker-selection','value'),
)

def update_dashboard(n_clicks,dashboard_selection,ticker_selection):
    fig = go.Figure()
    
    fig.update_layout(
        paper_bgcolor='#000',
        plot_bgcolor='#222',
        xaxis_gridcolor='#333',
        yaxis_gridcolor='#333',
        font=dict(size=11,color='#fff')
    )
    
    if n_clicks != 0:
        
        ticker_selection = ticker_selection.upper().strip()
        folder = 'cache'
        if not os.path.exists(folder):
            os.makedirs(folder)

        path = f'{folder}/{ticker_selection}_financials_quarterly.csv'

        if not os.path.exists(path):
            try:
                ticker = yf.Ticker(ticker_selection)
                stockname = ticker.info['longName']
                cf = ticker.get_cashflow(freq='quarterly')
                inc = ticker.get_incomestmt(freq='quarterly')
                df = pd.concat([cf,inc],axis=0,join='inner')
                df.columns = pd.to_datetime(df.columns).to_period('Q').astype('str')
                df[df.columns[::-1]].to_csv(path)
            except Exception as e:
                logging.error(e)
                return fig

        else:
            try:
                df = pd.read_csv(path,index_col=0)
            except Exception as e:
                logging.error(e)
                return fig
            
    ticker = yf.Ticker(ticker_selection)
    stockname = ticker.info['longName']
    
    try:
        if dashboard_selection == 'margin':

            netmargin = df.loc['NetIncome']/df.loc['TotalRevenue']*100
            fcfmargin = df.loc['FreeCashFlow']/df.loc['TotalRevenue']*100

            fig.add_trace(
                go.Bar(
                    x=netmargin.index,
                    y=netmargin,
                    marker=dict(color='royalblue',line=dict(color='#000',width=1)),
                    name='Net-Margin',
                    hovertemplate='<b>%{x}</b><br>%{y:.2f}%'
                )
            )
            fig.add_trace(
                go.Bar(
                    x=fcfmargin.index,
                    y=fcfmargin,
                    marker=dict(color='limegreen',line=dict(color='#000',width=1)),
                    name='FCF-Margin',
                    hovertemplate='<b>%{x}</b><br>%{y:.2f}%'
                )
            )
            fig.update_layout(
                title=dict(text=f'{stockname} Margins',font=dict(size=17,weight='bold')),
                yaxis=dict(title='Margin(%)',gridcolor='#333')
            )
            
        elif dashboard_selection == 'growth':

            rev = df.loc['TotalRevenue'].pct_change().fillna(0.0)*100
            ni = df.loc['NetIncome'].fillna(0.0).pct_change().fillna(0.0)*100

            fig.add_trace(
                go.Scatter(
                    x=ni.index,
                    y=ni,
                    name='NetIncome',
                    mode='lines+markers',
                    line=dict(color='gold',width=2),
                    marker=dict(size=11,symbol='diamond',line=dict(color='#000',width=1)),
                    hovertemplate='<b>%{x}</b><br>%{y:.2f}%'
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=rev.index,
                    y=rev,
                    name='TotalRevenue',
                    mode='lines+markers',
                    line=dict(color='red',width=2),
                    marker=dict(size=11,symbol='diamond',line=dict(color='#000',width=1)),
                    hovertemplate='<b>%{x}</b><br>%{y:.2f}%'
                )
            )
            fig.update_layout(
                title=dict(text=f'{stockname} Growth',font=dict(size=17,weight='bold')),
                yaxis=dict(title='Change(%)',gridcolor='#333'),
                plot_bgcolor='#222'
            )
    except Exception as e:
        logging.error(e)
        
    return fig


if __name__ == '__main__':
    app.run(debug=True)